In [1]:
# Trying to get hyperlinks for all states so we can run the scraping script on all states.

# Imports and get the html data from MountainProject. 
import pandas as pd
import numpy as np
import string
import requests
from bs4 import BeautifulSoup

url = 'https://www.mountainproject.com/route-guide'
page = requests.get(url)

soup = BeautifulSoup(page.content, 'html.parser')

# Put all hyperlinks on the page into a list.
data = []
for link in soup.find_all('a', class_= 'text-truncate float-xs-left'):
    data.append(link.get('href'))
    
# List of US States that we'll use to remove unwanted hyperlinks.
# Mississippi and Nebrasaka don't have pages, which makes sense,
# considering how flat they are.
states = ["Alabama","Alaska","Arizona","Arkansas","California","Colorado",
  "Connecticut","Delaware","Florida","Georgia","Hawaii","Idaho","Illinois",
  "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
  "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
  "Nebraska","Nevada","New-Hampshire","New-Jersey","New-Mexico","New-York",
  "North-Carolina","North-Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
  "Rhode-Island","South-Carolina","South-Dakota","Tennessee","Texas","Utah",
  "Vermont","Virginia","Washington","West-Virginia","Wisconsin","Wyoming"]

# List of US States and their abbreviations for a dictionary.
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New-Hampshire': 'NH',
    'New-Jersey': 'NJ',
    'New-Mexico': 'NM',
    'New-York': 'NY',
    'North-Carolina': 'NC',
    'North-Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Palau': 'PW',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode-Island': 'RI',
    'South-Carolina': 'SC',
    'South-Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West-Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}

# Make the states lowercase to match hyperlinks. 
states = [x.lower() for x in states]
us_state_abbrev = dict((k.lower(),v) for k,v in us_state_abbrev.items())

# Loop through each hyperlink and see if it contains a state as a subsrting.
# If it does, add the link to a list containing links for each state page. 
US = []
for datum in data:
    for state in states:
        if state in datum:
            US.append([state,datum])
        
# Remove duplicate links, Vermont Ice+Mixed, and Tennessee Wall (they are duplicates!)
# Check length. Should be 48. 
res = [] 
[res.append(x) for x in US if x not in res] 
# US=list(set(US))
res.remove(['tennessee','https://www.mountainproject.com/area/105851828/the-tennessee-wall'])
res.remove(['vermont','https://www.mountainproject.com/area/107280521/vermont-ice-and-mixed'])
links = dict(res)

In [25]:
def climb_dataframe(link, state):
    url = link
    page = requests.get(link)
    soup = BeautifulSoup(page.content,'html.parser')
    #dfClimb = pd.DataFrame()
    route_count = []
    route_type = []
    for climb_type in climb_types:
        for route in soup.find_all('span', class_ = "lef-nav-" + climb_type):
            route_count.append(route.text)
            route_type.append(climb_type)

    route_count = [s.strip() for s in route_count]
    route_count = [s.replace(' /','') for s in route_count]
    route_count = [s.replace(',','') for s in route_count]
    route_count = [int(s) for s in route_count]
    df = pd.DataFrame(data = route_count)
    df['type'] = route_type
    df = df.set_index('type')
    df = df.sum(level = 'type')
    #df.assign(ST = us_state_abbrev[state])
    df['ST'] = us_state_abbrev[state]
    return df

In [26]:
climb_types = ['Trad','Sport','Toprope','Boulder','Ice','Aid','Mixed','Alpine']
climb_df = pd.DataFrame()
for sta, link in links.items():
    climb_df = climb_df.append(climb_dataframe(link,sta))

In [27]:
climb_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 384 entries, Trad to Alpine
Data columns (total 2 columns):
0     384 non-null int64
ST    384 non-null object
dtypes: int64(1), object(1)
memory usage: 9.0+ KB


In [19]:
us_state_abbrev[state]

'WY'

In [29]:
climb_df.head(50)

,0,ST
type,,
Trad,249,AL
Sport,317,AL
Toprope,103,AL
Boulder,873,AL
Ice,0,AL
Aid,1,AL
Mixed,0,AL
Alpine,0,AL
Trad,807,AK


In [34]:
def test_climb_dataframe(link, state):
    url = link
    page = requests.get(link)
    soup = BeautifulSoup(page.content,'html.parser')
    #dfClimb = pd.DataFrame()
    route_count = []
    route_type = []
    for climb_type in climb_types:
        for route in soup.find_all('span', class_ = "lef-nav-" + climb_type):
            route_count.append(route.text)
            route_type.append(climb_type)

    route_count = [s.strip() for s in route_count]
    route_count = [s.replace(' /','') for s in route_count]
    route_count = [s.replace(',','') for s in route_count]
    route_count = [int(s) for s in route_count]
    df = pd.DataFrame(data = route_count)
    df['type'] = route_type
    #df = df.set_index('type')
    #df = df.sum(level = 'type')
    #df.assign(ST = us_state_abbrev[state])
    df['ST'] = us_state_abbrev[state]
    return df

In [35]:
climb_types = ['Trad','Sport','Toprope','Boulder','Ice','Aid','Mixed','Alpine']
test_df = pd.DataFrame()
for sta, link in links.items():
    test_df = test_df.append(test_climb_dataframe(link,sta))

In [37]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9120 entries, 0 to 255
Data columns (total 3 columns):
0       9120 non-null int64
type    9120 non-null object
ST      9120 non-null object
dtypes: int64(1), object(2)
memory usage: 285.0+ KB


In [38]:
test_df.head()

,0,type,ST
0,1,Trad,AL
1,7,Trad,AL
2,0,Trad,AL
3,10,Trad,AL
4,0,Trad,AL


In [39]:
len(test_df['type'])

9120

In [47]:
hope = test_df.set_index('ST')
hope.columns = ['num','type']
hope.head()

,num,type
ST,,
AL,1,Trad
AL,7,Trad
AL,0,Trad
AL,10,Trad
AL,0,Trad


In [41]:
faith = hope.sum(level = 'ST')

In [42]:
faith.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48 entries, AL to WY
Data columns (total 1 columns):
0    48 non-null int64
dtypes: int64(1)
memory usage: 768.0+ bytes


In [52]:
hope.head()

,num,type
ST,,
AL,1,Trad
AL,7,Trad
AL,0,Trad
AL,10,Trad
AL,0,Trad


In [54]:
table = pd.pivot_table(hope,values = 'num', columns = 'type', index ='ST', aggfunc = np.sum)

In [55]:
table

type,Aid,Alpine,Boulder,Ice,Mixed,Sport,Toprope,Trad
ST,,,,,,,,
AK,19,502,257,226,60,672,186,807
AL,1,0,873,0,0,317,103,249
AR,1,0,463,0,0,987,58,416
AZ,88,4,2148,0,0,4035,392,3536
CA,272,1022,10397,105,57,8364,3200,11240
CO,228,1425,7641,751,525,9707,1823,9542
CT,1,0,1064,3,1,79,503,564
DE,0,0,22,0,0,0,14,0
FL,0,0,15,0,0,0,1,0
